# Data Preperation

In [ ]:
import pandas as pd
import requests
import io
import gdown
import pickle

## Data Loading

In [2]:
def make_drive_url(file_id):
    return f"https://drive.google.com/uc?id={file_id}"

# Helper function to load a CSV from a direct URL
def load_csv_from_url(url):
    response = requests.get(url)
    response.raise_for_status()  # Raises an error if the request fails
    return pd.read_csv(io.StringIO(response.text))

# Dictionary of file IDs for clarity
file_ids = {
    "holiday_events": "1RMjSuqHXHTwAw_PGD5XVjhA3agaAGHDH",
    "items": "1ogMRixVhNY6XOJtIRtkRllyOyzw1nqya",
    "oil": "1Q59vk2v4WQ-Rpc9t2nqHcsZM3QWGFje_",
    "stores": "1Ei0MUXmNhmOcmrlPad8oklnFEDM95cDi",
    "train": "1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv",
    "transactions": "1PW5LnAEAiL43fI5CRDn_h6pgDG5rtBW_"
}

# Load each CSV using the helper functions
df_holiday_events = load_csv_from_url(make_drive_url(file_ids["holiday_events"]))
df_items          = load_csv_from_url(make_drive_url(file_ids["items"]))
df_oil            = load_csv_from_url(make_drive_url(file_ids["oil"]))
df_stores         = load_csv_from_url(make_drive_url(file_ids["stores"]))
df_transactions   = load_csv_from_url(make_drive_url(file_ids["transactions"]))

In [ ]:
# train.csv is too big (5 GB) so we download it separately

train_url = make_drive_url(file_ids["train"])
gdown.download(train_url, "../resources/train.csv", quiet=False)

### Filter Stores from "Guayas" Region

In [3]:
store_ids = df_stores[df_stores['state'] == 'Guayas']['store_nbr'].unique()
store_ids

array([24, 26, 27, 28, 29, 30, 32, 34, 35, 36, 51])

Filter stores in chunks

In [28]:
chunk_size = 10**6  # 1 million rows at a time
filtered_chunks = []

# Add dtype specifications to avoid warnings
dtype_specs = {
    'id': 'int64',
    'date': 'object',
    'store_nbr': 'int64',
    'item_nbr': 'int64',
    'unit_sales': 'float64',
}

for chunk in pd.read_csv('../resources/train.csv', chunksize=chunk_size, dtype=dtype_specs, low_memory=False):
    filtered_chunk = chunk[chunk['store_nbr'].isin(store_ids)]
    filtered_chunks.append(filtered_chunk)

    # free up memory
    del chunk

In [29]:
df_train = pd.concat(filtered_chunks, ignore_index=True)

In [30]:
df_train

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,18789,2013-01-02,24,103665,6.0,NaN
1,18790,2013-01-02,24,105574,12.0,NaN
2,18791,2013-01-02,24,105575,2.0,NaN
3,18792,2013-01-02,24,105577,1.0,NaN
4,18793,2013-01-02,24,105693,7.0,NaN
...,...,...,...,...,...,...
22941651,125491520,2017-08-15,51,2123727,2.0,False
22941652,125491521,2017-08-15,51,2123750,3.0,False
22941653,125491522,2017-08-15,51,2123775,1.0,False
22941654,125491523,2017-08-15,51,2123791,1.0,False


In [31]:
df_train.shape

(22941656, 6)

In [32]:
# free memory
del filtered_chunks

### Keeping only the three biggest product families

In [33]:
df_top3_product_families = df_items['family'].value_counts().head(3).reset_index()
df_top3_product_families

,family,count
0,GROCERY I,1334
1,BEVERAGES,613
2,CLEANING,446


In [34]:
item_ids_top3 = df_items[df_items['family'].isin(df_top3_product_families['family'])]['item_nbr'].unique()
item_ids_top3

array([  96995,   99197,  103501, ..., 2132945, 2132957, 2134058])

In [35]:
df_train = df_train[df_train['item_nbr'].isin(item_ids_top3)]
df_train

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
1,18790,2013-01-02,24,105574,12.0,NaN
2,18791,2013-01-02,24,105575,2.0,NaN
3,18792,2013-01-02,24,105577,1.0,NaN
4,18793,2013-01-02,24,105693,7.0,NaN
5,18794,2013-01-02,24,105737,17.0,NaN
...,...,...,...,...,...,...
22941650,125491519,2017-08-15,51,2122188,1.0,False
22941651,125491520,2017-08-15,51,2123727,2.0,False
22941652,125491521,2017-08-15,51,2123750,3.0,False
22941653,125491522,2017-08-15,51,2123775,1.0,False


### Agrregate Sales for each day

In [36]:
df_filtered = df_train.groupby('date').sum()['unit_sales'].reset_index()

### Fill missing Days with Zero Sales

In [37]:
df_filtered['date'] = pd.to_datetime(df_filtered['date'])
df_filtered.set_index('date', inplace=True)
df_filtered = df_filtered.asfreq('D').fillna(0)

### Save df_train as pickle

In [39]:
dataframes = {
    'df_holiday_events': df_holiday_events,
    'df_items': df_items,
    'df_oil': df_oil,
    'df_stores': df_stores,
    'df_transactions': df_transactions,
    'df_train': df_train,
    'df_filtered': df_filtered
}

with open('../resources/1_dataframes.pkl', 'wb') as f:
    pickle.dump(dataframes, f)